# Use Gen AI to build DENG pipeline 

This presents the idea for how to use GenAI, to help to build a medallion architecture

## Basic Setup
We'll use [duckdb](https://duckdb.org/) as the database.

In [1]:
import duckdb
# import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Config
# %config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True
%config SqlMagic.displaylimit = None

# Connect to DuckDB
# %sql duckdb:///:default:
# %sql duckdb:///:memory:
%sql duckdb:////Users/jesse23/u/p/tide/notes/test.db

displaylimit: Value None will be treated as 0 (no limit)

Assuming we have source data as below:

In [2]:
%%sql 
-- part data, assuming import by a CSV
CREATE TABLE IF NOT EXISTS bronze_design
  (id INTEGER, name VARCHAR, type VARCHAR, designer VARCHAR);
INSERT INTO bronze_design VALUES 
  (0,  'part0',   'part', 'Allen'), 
  (1,  'part1',   'part', 'Bobby'), 
  (2,  'part2',   'part', 'Cindy'), 
  (3,  'part00',  'part', 'David'), 
  (4,  'part001', 'part', 'Eddie'), 
  (5,  'part002', 'part', 'Frank'), 
  (6,  'part10',  'part', 'George'), 
  (7,  'part100', 'part', 'Harry'), 
  (8,  'part101', 'part', 'Ivan'), 
  (9,  'part11',  'part', 'Jack'), 
  (10, 'part110', 'part', 'Kevin'), 
  (11, 'part111', 'part', 'Larry'); 

-- company info, import by a CSV
CREATE TABLE IF NOT EXISTS bronze_design_office
  (design VARCHAR, office VARCHAR);
INSERT INTO bronze_design_office VALUES 
  ('part001', 'SmartView' ), 
  ('part10', 'FutureIdea'); 

SELECT * from bronze_design;

id,name,type,designer
0,part0,part,Allen
1,part1,part,Bobby
2,part2,part,Cindy
3,part00,part,David
4,part001,part,Eddie
5,part002,part,Frank
6,part10,part,George
7,part100,part,Harry
8,part101,part,Ivan
9,part11,part,Jack


In [3]:
%%sql
-- Step 1: Create a mapping table to hold the UUIDs for each design
CREATE TABLE IF NOT EXISTS design_uuid_map AS
SELECT id, name, type, designer, uuid() AS uuid
FROM bronze_design;

-- Step 2: Create a mapping for parents and children
CREATE TABLE IF NOT EXISTS design_parent_child AS
SELECT 
    child.id AS child_id,
    parent.uuid AS parent_uuid
FROM 
    bronze_design child
LEFT JOIN 
    design_uuid_map parent
ON 
    parent.name = SUBSTRING(child.name FROM 1 FOR LENGTH(child.name) - 1);

-- Step 3: Merge the data to create the silver table
CREATE TABLE silver_design AS
SELECT 
    d.uuid AS id,
    d.name,
    d.type,
    d.designer,
    o.office,
    pc.parent_uuid
FROM 
    design_uuid_map d
LEFT JOIN 
    bronze_design_office o
ON 
    d.name = o.design
LEFT JOIN 
    design_parent_child pc
ON 
    d.id = pc.child_id;


Count


In [4]:
%%sql
select * from silver_design;

id,name,type,designer,office,parent_uuid
21803e18-c9ed-48f5-9b34-5e6b949f9ad3,part001,part,Eddie,SmartView,ab21bde4-7baa-48bd-99e7-17482b5856c7
32d6afa0-8c92-4424-81ea-8b97597bcc54,part10,part,George,FutureIdea,f5c222db-efe5-431b-8d5f-558a3ee3fbf3
4ffa0d82-11b1-45cd-a5fa-c54007a4a487,part0,part,Allen,None,None
f5c222db-efe5-431b-8d5f-558a3ee3fbf3,part1,part,Bobby,None,None
33e3ecd7-4d20-4d0a-a86a-cbfd5c8fec2c,part2,part,Cindy,None,None
ab21bde4-7baa-48bd-99e7-17482b5856c7,part00,part,David,None,4ffa0d82-11b1-45cd-a5fa-c54007a4a487
b5908dc7-4a4a-464b-8f2f-9cf0ee3e1a56,part002,part,Frank,None,ab21bde4-7baa-48bd-99e7-17482b5856c7
bca3aea1-0343-4e23-858a-00d4d6ccfb37,part100,part,Harry,None,32d6afa0-8c92-4424-81ea-8b97597bcc54
b926df8a-3e92-418f-beb6-673f57af17d3,part101,part,Ivan,None,32d6afa0-8c92-4424-81ea-8b97597bcc54
3e6c17b4-e3f7-47c4-b9d3-edf34db4816c,part11,part,Jack,None,f5c222db-efe5-431b-8d5f-558a3ee3fbf3
